In [1]:
from dotenv import load_dotenv
load_dotenv()

from pydantic import BaseModel, Field

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

from src.path import DATA_DIR

# Load Saved VectorDB

In [2]:
DB_DIR = DATA_DIR / "DB"

embedding = OpenAIEmbeddings(model = "text-embedding-3-small")

db = Chroma(persist_directory=DB_DIR, embedding_function = embedding)

# Simple Method

In [3]:
query = "거주자가 뭐야?"

In [4]:
retrieved_docs = db.similarity_search(query, k = 2)

In [5]:
message = f"""
    질문에 대한 답변을 작성할 때, 리트리버에서 가져온 문서를 참고하여 답변을 작성하세요.

    질문: 
    {query}

    참고: 
    {retrieved_docs}
"""

In [6]:
print(message)


    질문에 대한 답변을 작성할 때, 리트리버에서 가져온 문서를 참고하여 답변을 작성하세요.

    질문: 
    거주자가 뭐야?

    참고: 
    [Document(id='0e2abab3-fd4c-4cfe-a0d9-b6e3f1df2c27', metadata={'source': 'd:\\langchain_2\\data\\소득세법_1.docx'}, page_content='1. “거주자”란 국내에 주소를 두거나 183일 이상의 거소(居所)를 둔 개인을 말한다.\n\n2. “비거주자”란 거주자가 아닌 개인을 말한다.\n\n3. “내국법인”이란 「법인세법」 제2조제1호에 따른 내국법인을 말한다.\n\n4. “외국법인”이란 「법인세법」 제2조제3호에 따른 외국법인을 말한다.\n\n5. “사업자”란 사업소득이 있는 거주자를 말한다.\n\n② 제1항에 따른 주소ㆍ거소와 거주자ㆍ비거주자의 구분은 대통령령으로 정한다.\n\n[본조신설 2009. 12. 31.]'), Document(id='52c7b644-755a-4b62-b989-b01aff43b9d7', metadata={'source': 'd:\\langchain_2\\data\\소득세법_1.docx'}, page_content='② 거주자가 사망한 경우의 과세기간은 1월 1일부터 사망한 날까지로 한다.\n\n③ 거주자가 주소 또는 거소를 국외로 이전(이하 “출국”이라 한다)하여 비거주자가 되는 경우의 과세기간은 1월 1일부터 출국한 날까지로 한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제6조(납세지) ① 거주자의 소득세 납세지는 그 주소지로 한다. 다만, 주소지가 없는 경우에는 그 거소지로 한다.')]



In [7]:
llm = ChatOpenAI(model_name = "gpt-5-nano")

In [8]:
response = llm.invoke(message)

In [8]:
print(response.content)

거주자란 국내에 주소를 두거나 183일 이상 거소를 둔 개인을 말합니다. 비거주자는 거주자가 아닌 사람이고, 거주자 여부와 주소/거소의 구분은 대통령령으로 정합니다. 또한 거주자의 납세지는 주소지이며, 주소지가 없으면 거소지로 정합니다. 

참고: 소득세법 제1조 제1항의 정의와 제6조의 납세지 규정에 근거한 내용입니다.


In [12]:
# without RAG
msg = message = f"""
    질문에 대한 답변을 간단하게 작성해줘.

    질문: 
    {query}
"""

rsp = llm.invoke(msg)
print(rsp.content)

거주자란 어떤 곳에 실제로 살고 그곳을 거주지로 삼아 생활하는 사람을 말합니다. 법적 맥락에 따라 다르게 정의될 수 있지만, 일반적으로는 그 장소에 주소나 본거지를 두고 장기간 거주하는 사람을 뜻합니다.


# Function Method

In [ ]:
def get_db():
    DB_DIR = DATA_DIR / "DB"

    embedding = OpenAIEmbeddings(model = "text-embedding-3-small")

    db = Chroma(persist_directory=DB_DIR, embedding_function = embedding)
    return db
    

def get_llm(model: str = "gpt-5-nano"):
    llm = ChatOpenAI(model_name = model)
    return llm

def generate_response(query, k: int = 2):
    db = get_db()
    llm = get_llm()
    
    retrieved_docs = db.similarity_search(query, k = k)

    message = f"""
        질문에 대한 답변을 작성할 때, 리트리버에서 가져온 문서를 참고하여 답변을 작성하세요.

        질문: 
        {query}

        참고: 
        {retrieved_docs}
    """

    response = llm.invoke(message)
    return response.content


In [10]:
print(generate_response(query))

- 거주자란: 국내에 주소를 두거나 183일 이상 거소를 둔 개인을 말합니다. (소득세법 제1항)
- 비거주자란: 거주자가 아닌 개인을 말합니다.
- 참고로 이 구분은 대통령령으로 정합니다.
- 세금 맥락에서의 납세지는 거주자의 주소지이며, 주소지가 없으면 거소지로 정합니다. (제6조)


# LCEL Method

LCEL(LangChain Expression Language)은 랭체인의 구성 요소들을 조합하여 복잡한 로직을 만들 수 있게 하는 메서드입니다.

'|' 연산자로 각 객체를 편리하게 조합할 수 있습니다.

EX) chain = component1 | component2 | component3

LCEL을 사용하면 다음과 같은 이점을 얻을 수 있습니다. 

- 가독성: 코드가 데이터의 흐름을 그대로 보여줍니다.
- 재사용성: 만든 체인은 다른 체인의 일부로 사용할 수 있습니다.

## PromptTemplate, OutputParser

### PromptTemplate

랭체인에서는 프롬프트를 객체로 관리하도록 PromptTemplate를 제공합니다.

변수를 포함하는 템플릿을 정의하고 실행 시점에 변수의 값들을 실젯값으로 채워넣어서 프롬프트를 완성할 수 있게 해줍니다.

대표적인 PromptTemplate의 유형은 아래와 같습니다.
- PromptTemplate: 가장 기본적인 템플릿. 단일 문자열 프롬프트 생성.
- ChatPromptTemplate: 채팅 모델을 위한 템플릿. 메시지 객체(SystemMessage, HumanMessage 등)의 리스트 생성.
- FewShotPromptTemplate: 모델에게 작업 수행 방식을 보여주는 몇 가지 예시를 프롬프트에 동적으로 포함시키고자 할 때 사용.
- PipelinePromptTemplate: 여러 프롬프트 템프릿을 순차적으로 연결하여 최종 프롬프트를 구성

PromptTemplate 객체 생성 방법
- from_template(): 문자열로 바로 생성
- 생성자 호출: 변수 목록, 템플릿을 명시하여 세밀한 제어가 가능
- load_prompt(): 파일에서 템플릿을 불러 올 수 있음
- partial_variables: 일부 변수를 고정하여 하위-프롬프트 생성가능

In [14]:
template = PromptTemplate(
    input_variables = ["question", "context"],
    template = """
아래 참고 문서를 기반으로 질문에 대한 답변을 간단하게 작성하세요.

질문:
{question}

참고 문서:
{context}
"""
)

In [15]:
print(template.format(question = "거주자가 뭐에요?", context = "거주자란 ~입니다."))


아래 참고 문서를 기반으로 질문에 대한 답변을 간단하게 작성하세요.

질문:
거주자가 뭐에요?

참고 문서:
거주자란 ~입니다.



### OutputParser

LLM은 기본적으로 텍스트 문자열을 반환합니다.

OutputParser는 이 텍스트 응답을 애플리케이션에서 사용하기 더 편리한 구조의 데이터(JSON, 리스트, Pydantic모델 객체 등)으로 변환하는 역할을 합니다.

또한, 응답의 파싱 과정에서 발생하는 오류를 처리하거나 LLM이 잘못된 형식으로 응답하는 경우 이를 수정하도록 시도할 수 있습니다.

OutputParser의 종류는 아래와 같습니다.
- StrOutputParser: 가장 기본적인 파서. 모델의 출력을 문자열로 반환
- SimpleJsonOutputParser: LLM의 응답이 간단한 JSON 문자열일 것으로 예상될 때 사용. JSON 문자열을 파이썬 딕셔너리로 변환
- PydanticOutputParser: Pydantic 모델을 사용하여 LLM의 응답을 파이썬 객체로 변환. 데이터 유효성 검사 및 타입 힌트의 이점을 누릴 수 있음
- CommaSeperatedListOutputParser:LLM이 쉼표로 구분된 리스트 형태의 문자열을 반환할 것으로 예상될 때 사용. 문자열을 리스트로 변환
- DatetimeOutputParser: 날짜/시간 정보를 포함하는 문자열을 반환할 때 이를 파이썬의 datetime 객체로 파싱
- XMLOutputParser: LLM이 XML 형식으로 응답하도록 유도하고 파싱
- MarkdownOutputParser: LLM이 마크다운으로 응답할 때 활용

In [43]:
# stroutputparser

chat_model = ChatOpenAI(model = "gpt-5-nano")

template = PromptTemplate(
    input_variables = ["question"],
    template = """
당신은 까칠한 AI 도우미입니다. 사용자의 질문에 최대 3줄로 답하세요.

질문:
{question}
"""
)

string_output_parser = StrOutputParser()


In [44]:
chain = template | chat_model | string_output_parser


In [45]:
result= chain.invoke({"question": "영화 '기생충'에 대한 리뷰를 작성해줘"})


In [46]:
print(result)

기생충은 빈부 격차를 서스펜스와 풍자로 찔러넣는, 장르를 넘나드는 걸작이다.
집과 계층의 공간 구성이 마치 캐릭터처럼 움직여 매 장면이 예리한 풍자를 남긴다.
엔딩이 다소 과장될 때도 있지만, 우리 사회를 냉정하게 들여다보게 만드는 강력한 한편이다.


In [ ]:
# JSON 출력

class MovieReviewTemplate(BaseModel):
    """영화 답변 스키마 정의 """
    title: str = Field(description = "영화 제목")
    rating: float = Field(description="10점 만점 평점 (예: 7.5)")
    review: str = Field(description = "한글 리뷰 (3~4문장)")

struncured_llm = chat_model.with_structured_output(MovieReviewTemplate)

chain = template | struncured_llm


In [61]:
result= chain.invoke({"question": "영화 '기생충'에 대한 리뷰를 작성해줘"})

In [62]:
print(type(result))

<class '__main__.MovieReviewTemplate'>


In [66]:
print(result)

title='기생충' rating=8.5 review='까칠한 요약: 봉준호의 기생충은 계층 간 불평등을 예리하게 짚는 냉소적이면서도 매끈한 가족 스릴러다.\n장치와 반전이 강력하고 현장감 있는 연출로 몰입을 끌어올리지만, 메시지가 다층이라 한 번에 이해하기는 어렵다.\n그럼에도 사회풍자와 서스펜스의 조합은 여운을 남기며, 생각할 거리를 확실히 남기는 걸작이다.'


In [63]:
print(result.title)

기생충


In [64]:
print(result.rating)

8.5


In [65]:
print(result.review)

까칠한 요약: 봉준호의 기생충은 계층 간 불평등을 예리하게 짚는 냉소적이면서도 매끈한 가족 스릴러다.
장치와 반전이 강력하고 현장감 있는 연출로 몰입을 끌어올리지만, 메시지가 다층이라 한 번에 이해하기는 어렵다.
그럼에도 사회풍자와 서스펜스의 조합은 여운을 남기며, 생각할 거리를 확실히 남기는 걸작이다.


In [69]:
template = PromptTemplate(
    input_variables = ["question", "context"],
    template = """
아래 참고 문서를 기반으로 질문에 대한 답변을 간단하게 작성하세요.

질문:
{question}

참고 문서:
{context}
"""
)

In [70]:
retriever = db.as_retriever()

In [71]:
chat_model = ChatOpenAI(model = "gpt-5-nano")

In [72]:
string_output_parser = StrOutputParser()

In [75]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [76]:
chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | template
    | chat_model
    | string_output_parser
)

In [77]:
response = chain.invoke("거주자에 대해 알려줘.")

In [78]:
print(response)

- 정의: 거주자란 국내에 주소를 두거나 183일 이상 거소를 둔 개인을 말합니다.
- 비거주자: 거주자가 아닌 개인입니다.
- 납세지: 거주자의 소득세 납세지는 주소지이며, 주소가 없으면 거소지로 정합니다.
- 구분의 근거: 거주자/비거주자 및 주소와 거소의 구분은 대통령령으로 정합니다.
